In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
Book_Ratings=pd.read_csv('BX-Book-Ratings.csv',sep=";",encoding="latin-1",on_bad_lines='skip')

In [4]:
Books=pd.read_csv('BX-Books.csv',sep=";",encoding="latin-1",on_bad_lines='skip')

C:\Users\278bs\AppData\Local\Temp\ipykernel_8736\295882231.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  Books=pd.read_csv('BX-Books.csv',sep=";",encoding="latin-1",on_bad_lines='skip')


In [5]:
Users=pd.read_csv('BX-Users.csv',sep=";",encoding="latin-1",on_bad_lines='skip')

In [6]:
Book_Ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [7]:
Book_Ratings.shape

(1149780, 3)

In [8]:
Users.shape

(278858, 3)

In [9]:
Books.shape

(271360, 8)

In [10]:
User_that_more_then200=Book_Ratings['User-ID'].value_counts()[Book_Ratings['User-ID'].value_counts() > 200].reset_index()


In [11]:
Book_Ratings=pd.merge(Book_Ratings,User_that_more_then200,on="User-ID")

In [12]:
Book_Ratings.shape

(526356, 4)

In [13]:
Books.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1, inplace=True)


In [14]:
Book_Ratings_for_books=pd.read_csv('BX-Book-Ratings.csv',sep=";",encoding="latin-1",on_bad_lines='skip')

In [15]:
Book_Ratings_for_books.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [16]:
book_that_more_then50=Book_Ratings_for_books['ISBN'].value_counts()[Book_Ratings_for_books['ISBN'].value_counts() > 100].reset_index()


In [17]:
book_that_more_then50.head()

,ISBN,count
0,0971880107,2502
1,0316666343,1295
2,0385504209,883
3,0060928336,732
4,0312195516,723


In [18]:
Book_count_merge=pd.merge(Books,book_that_more_then50,on="ISBN",how="inner")

In [19]:
Book_count_merge.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,count
0,0440234743,The Testament,John Grisham,1999,Dell,422
1,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison,1994,Plume,180
2,0971880107,Wild Animus,Rich Shapero,2004,Too Far,2502
3,0345402871,Airframe,Michael Crichton,1997,Ballantine Books,207
4,0345417623,Timeline,MICHAEL CRICHTON,2000,Ballantine Books,407


In [20]:
final_ratings=pd.merge(Book_count_merge,Book_Ratings,on="ISBN",how="inner")

In [21]:
final_ratings.shape

(48990, 9)

In [22]:
final_ratings.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,count_x,User-ID,Book-Rating,count_y
0,0440234743,The Testament,John Grisham,1999,Dell,422,277478,0,214
1,0440234743,The Testament,John Grisham,1999,Dell,422,2977,0,232
2,0440234743,The Testament,John Grisham,1999,Dell,422,3363,0,901
3,0440234743,The Testament,John Grisham,1999,Dell,422,7346,9,1007
4,0440234743,The Testament,John Grisham,1999,Dell,422,9856,0,201


In [25]:
final_ratings=final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')


In [26]:
final_ratings.fillna(0,inplace=True)


In [27]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_scores = cosine_similarity(final_ratings)
similarity_scores.shape


(664, 664)

In [28]:
from scipy.sparse import csr_matrix 
book_sparse=csr_matrix(final_ratings)


In [29]:
type(book_sparse)


scipy.sparse._csr.csr_matrix

In [31]:
from sklearn.neighbors import NearestNeighbors 
model=NearestNeighbors(algorithm='brute')
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [34]:
final_ratings.iloc[237]

User-ID
254       0.0
2276      0.0
2766      0.0
2977      0.0
3363      0.0
         ... 
275970    0.0
277427    0.0
277478    0.0
277639    0.0
278418    0.0
Name: How to Make an American Quilt, Length: 882, dtype: float64

In [39]:
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "4"  # Set the number of cores you want to use


In [40]:
input_data = final_ratings.iloc[237, :].values.reshape(1, -1)
distances, indices = model.kneighbors(input_data, n_neighbors=6)


In [42]:
indices

array([[237, 162, 344, 276, 260, 194]], dtype=int64)

In [45]:
for i in range(len(indices)):
    print(final_ratings.index[indices[i]])

Index(['How to Make an American Quilt', 'Eaters of the Dead',
       'Pleading Guilty', 'Last Man Standing', 'Jackdaws', 'Four Blondes'],
      dtype='object', name='Book-Title')


In [46]:
final_ratings.index[237]

'How to Make an American Quilt'